## 3.1.1 Tấn công sử dụng tính căn bậc e

Bài toán

In [149]:
from Crypto.Util.number import long_to_bytes, bytes_to_long, getPrime, inverse

flag = b'Hello, World!'

p = getPrime(512)
q = getPrime(512)

n = p*q
phi = (p - 1) * (q - 1)
e = 3
d = inverse(e, phi)
c = pow(bytes_to_long(flag), e, n)

print('n =', hex(n))
print('e =', hex(e))
print('c =', hex(c))

n = 0x943006849b3ad8bcb730d7452e6a9f680fc699580debd151f0b9def29e693a585e1a300fd4ae2a07f05bac20d15f5dd60299121006b0fa90700ffce841c069c38c910c263fb2c07256dbbf39cca02bd4e0ae048055080211c1a7610939a73058eaf2602ef0c01cf6e6f1f2ed3a7051124bb3ac3828364be28188960909d3a689
e = 0x3
c = 0x5ca337215068974823c0e00eeefc194fd2734374eacc18e1512c8a91565ea40ed5b53b870b861


Lời giải

In [138]:
# ref: https://gist.github.com/jl4730/47c6c88bef60ac35b9da961dc424fe82
def find_invpow(x: int, n: int):
    # build because the math.log function does not work for super big numbers
    # the method leveraged binary search algorithm
    high = 1
    while high ** n <= x:
        high *= 2
    low = high // 2  # critical to use // instead of / because Python's integers are dynamically sized but not float
    while low < high:
        mid = (low + high) // 2
        if low < mid and mid ** n < x:
            low = mid
        elif high > mid and mid ** n > x:
            high = mid
        else:
            return mid
    return mid + 1

m = find_invpow(c, e)
print(long_to_bytes(m))

b'Hello, World!'


## 3.1.2 Tấn công Hastad Broadcast

Bài toán

In [17]:
from Crypto.Util.number import long_to_bytes, bytes_to_long, getPrime, inverse
from sympy.ntheory.modular import crt
def RSAKeyGen():
    e = 7
    p = getPrime(128)
    q = getPrime(128)
    n = p * q
    phi = (p-1) * (q-1)

    try:
        d = inverse(e, phi)
    except:
        return RSAKeyGen()
    
    return n, e #, d, phi

def enc():
    flag = b'Hello, World!'
    m = bytes_to_long(flag)
    n, e = RSAKeyGen()
    c = pow(m, e, n)

    return c, n, e


Lời giải

In [19]:
e = 7
ns = []
cs = []
for i in range(e):
    c, n, e = enc()
    cs.append(c)
    ns.append(n)

In [20]:
[hex(c) for c in cs]

['0x7b39a6dc1c4da2aba3cfadf5e045fbe31a064f2b44326d8408ca8d53bd1582f8',
 '0x55e7091158f372f9777d337fd65adc015a55eb11f774464ae614aeb55c948f54',
 '0x49aa4adede1cc03a1a2f97fdee38b00c333fcdbad487e3415e920148c99e72a1',
 '0x6bd991da864a592a38e067b0f2212ab15728c5b73869717eae87d86273a7ef5e',
 '0xd5fc5e6c880737ac42490d8fa65e89bacefed236fdbdc6c3dd7cedf554ac8e',
 '0xd3a3d6b0592b2567110305369afa0381bea3a1d8988bdaf6bde73ca0c7b8dc9d',
 '0x67f44953ff33cc2471313b9cc3ca9e53219a6a6e36180736519778d7c3f0b49a']

In [21]:
[hex(n) for n in ns]

['0x996155318d2506b7afa4977ef7a1ff436b73f6a4194ae2306167c3b20c10fa1d',
 '0xbc78b5ccb113fe58749e496b39f8af73aac5e04fc4e6ba416e4ea0d50bc406fb',
 '0x84bb1da6bc942839855240929c4c6d46200c0a63d8b4fbafb6c61b10e13a29f1',
 '0xb1f5dcf1e17ebcd6aceda43c2fcadeff952978874e824b5a94f4215fed2a92b3',
 '0x833118500bc9eaa20698dd670786df5444052c3067b842476dd4c4db552564f5',
 '0xe2a937b7f5dd6a1b68fabba317bf64b9a368af0605ddf9fc3b05a29de02cce1d',
 '0xaf10c56424486718ebd953c46157ab7ff57c13e2a51ad838ecae38499f4f88f3']

In [22]:
me = crt(ns, cs)[0]
print(long_to_bytes(find_invpow(me, e)))

b'Hello, World!'

## 3.2 Tấn công Wiener

Tham khảo: https://github.com/pablocelayes/rsa-wiener-attack.git

Bài toán

In [38]:
import sys
sys.path.append('./rsa-wiener-attack')
import RSAwienerHacker as wa
import RSAvulnerableKeyGenerator as wg

from Crypto.Util.number import long_to_bytes, bytes_to_long

e, n, d = wg.generateKeys(256)

flag = b'Hello, World!'
c = pow(bytes_to_long(flag), e, n)

print('e =', hex(e))
print('n =', hex(n))
print('c =', hex(c))

e = 0xb4189834f2ce48483951d84cf848e3aeb0dfb1b3c12329b50fe0b99121800037
n = 0x2b4f1eb8c9ab9bf9c6bbf8cdf095bdaa49c67e35bf8487c64b9adeaff2f4d851f
c = 0xf6283682e03c5434f3142b9c3e79bda13c6ed3d7802ba96596bd88c1756d31e8


Lời giải

In [40]:
d = wa.hack_RSA(e, n)
print('d =', d)
m = pow(c, d, n)
print(long_to_bytes(m))

Hacked!
d = 7091840790055978951
b'Hello, World!'


## 3.3 Phân tích Fermat

Bài toán

In [57]:
from sympy import nextprime, primefactors
from Crypto.Util.number import getPrime
p = getPrime(2048)
q = nextprime(p)
n = p * q
print(hex(n))

0x86cafa12fab4afeaeb86ff371627e09646af4b1fffd844646ef051488c7e20711cc6982fc58b1b83179d01b826c2c0092515b0d75c6e12c8376555b6e45aa1ac8de02f9d86a90642e133e27f5cc7ce3b64fde0b8f8f4389eb6c04177990c3c121bc2b85056a9f0626d580267db26820f1baf2cd34809e9dda86c6468f4e7437770929ad1f9e8a1f0c60fea3817178576815a93b82a5d882deffe520dfedde06abfc0504f01b2da4a20289fa95877ccfc7e5a40b5aeccaf005b80d528e46d9617c6f94719fd583ba35b95f35e3a08f6f5d15bbfcd2d29e94d7457448c2ab4b9dbda26e204889acaa909dee6faca6f11ff5b945b1af3bb1cf5553cd239a7a6d70b6fe53e3e2f2a51a0ecc7b9d20741fb3e74c0ab48ff7b8dc285e57a13f4a38baa9f3edc5bb4257fb6536504f98dc91d2a793aaf9d9fda793740101116d1583d35aa04ac7c42eca8356e3ba836877ac47d548b3f573e4d64d1ab7dc7e031a54edd2b49804b2b08ef1df656a34ea0b5a8bafe3408ecf0199c3d79bfc5cbb38708bf34b6acebdb8a4e57081dc63d4d1438d8ca962309bb9e729c30b9f2ec075eab081b51aa52c3caebffaa96790e9e35ad5f383f3059d6a3cab94259b31091076db596d8fed3bf17d32b1db71f3d94129967b8bc0533f22029bd05c009d9ff9bde56113199bc03da43674c58f604c41cc8923ee3be

Lời giải

In [61]:
[p, q] = primefactors(n)
print('p =', hex(p))
print('q =', hex(q))

p = 0xb9c2b2820918c70800c8807f773e1003fdbe850d72ca33fcbffc86b0cfe0ec986d4d51b67230f6997c82cd0d7d5d2de0a7a8f520a60e198841063087f63d465f16e88563e002b7899ef12db928006c5c9d9fc4f0b6c3681e7de9261342e927a09d7ee2136c73e681c133e755f92e734fae82fe002f89355f670b569abd504ae4e6913af4a94a5dfcadcf6d876857fe80e52727cc5a79385d525412f6e63f2bd0fb2d1574dbfd150e6c24f55398b780dde9838094261c063e3bb7947811d2799feb58d9d70e3dafc980fb6e44ef9dabcbfd89f6fec2e1af74936d72a9dd698032d6b3031e7cff7a053ae0b0e51c9a428858ecc4f34dc8c90d1b9d84becf6326c1
q = 0xb9c2b2820918c70800c8807f773e1003fdbe850d72ca33fcbffc86b0cfe0ec986d4d51b67230f6997c82cd0d7d5d2de0a7a8f520a60e198841063087f63d465f16e88563e002b7899ef12db928006c5c9d9fc4f0b6c3681e7de9261342e927a09d7ee2136c73e681c133e755f92e734fae82fe002f89355f670b569abd504ae4e6913af4a94a5dfcadcf6d876857fe80e52727cc5a79385d525412f6e63f2bd0fb2d1574dbfd150e6c24f55398b780dde9838094261c063e3bb7947811d2799feb58d9d70e3dafc980fb6e44ef9dabcbfd89f6fec2e1af74936d72a9dd698032d6b3031e7cff7a053ae0b0e51c9

# 3.4.1 Sử dụng tính đồng cấu của hệ mã RSA

Bài toán

In [6]:
from Crypto.Util.number import getPrime, bytes_to_long, long_to_bytes
import gmpy2

flag = b'Hello, World!'
def get_keys():
    while(True):
        p = getPrime(128)
        q = getPrime(128)
        try:
            d = gmpy2.invert(65537, (p - 1) * (q - 1))
            return p, q

def enc(m):
    return pow(m, e, N)

def dec(c):
    return pow(c, d, N)


e = 65537
p, q = get_keys()
d = gmpy2.invert(e, (p - 1) * (q - 1))
N = p * q

c = enc(bytes_to_long(flag))
print('c =', hex(c))


c = 0x3622b9847197630578dbe545537c8f9250384a4b2abcb40375e4a7a2fb2df247


Lời giải

In [7]:
c1 = enc(2); print(c1)
c2 = c * c1; m2 = dec(c2)
m = (m2 * gmpy2.invert(2, N)) % N
print(long_to_bytes(m))

2667940651152658288875277051102719108207725823296805779533243811316118115881
b'Hello, World!'


# 3.4.2 Tấn công LSB

Bài toán

In [13]:
import Crypto
from Crypto.Util.number import bytes_to_long, long_to_bytes, getPrime
import gmpy2
import random

m = bytes_to_long(b'Hello, World!')
e = 65537

def genKey(size):
    while True:
        p = getPrime(size)
        q = getPrime(size)
        try:
            d = gmpy2.invert(65537, (p - 1) * (q - 1))
            return p, q
        except:
            continue

p, q = genKey(512)
N = p * q
phi = (p - 1) * (q - 1)
d = gmpy2.invert(e, phi)

def encrypt(m):
    return pow(m, e, N)

def partial_decrypt(c):
    return long_to_bytes(pow(c, d, N) & 255)

c = encrypt(m)
print(f'''c = {hex(c)}''')

c = 0x61e320ce2a717da4338537e8a10ec0c06b138880c0b6a42e12242f4308fbea67095c41f9d433fc1b5e3875b361b57ba6e2008d65bdbbbaa02f147ff277395aa33eaed628441c3b4727173ccade847d2b092c60baa01c60c984acc81c35b1f186c2804ae76c27b147d74bee8ded95ea957c591f81eb47eedc18eef3b131b703a9


Lời giải

In [14]:
partial_decrypt(c)

b'!'

# 3.5 Tấn công sử dụng phương pháp Coppersmith
# 3.5.1 Biết phần lớn bản tin gốc và bản mã hóa của nó, tìm phần còn lại của bản tin gốc

Bài toán

In [1]:
import Crypto
from Crypto.Util.number import bytes_to_long, long_to_bytes
import gmpy2
import random

flag2 = b'This is the flag: {Hello, World!} Good luck!'
m = bytes_to_long(flag2)
N = int('0x6f7d4b589de9bd9171ab941c46571de75bd73ab9f03d22fd21e71b206298c35e7a1dfc551b3f4b2f468c0dd6cc3bd4a471d95e6e7df85331', 16)
e = 3
c = pow(m, e, N)
print(f'c = {hex(c)}')

c = 0x8c1f34fd6ca9376bed1cdf7457e9a82f18698990ec9bbe6f832680b1bb092899f6ad0df0d97e1cc670b03e8f18961ec3d31953d734cc5b7


Lời giải

In [2]:
import Crypto
from Crypto.Util.number import bytes_to_long, long_to_bytes
import gmpy2
import random

U = bytes_to_long(b'This is the flag: {')
V = bytes_to_long(b'} Good luck!')

In [3]:
F.<x> = PolynomialRing(Zmod(N), implementation='NTL')
inv = gmpy2.invert(1 << 96, N)
known = int(U * (1 << 104) + V * inv)
inv2 = inv ** 3
f = (x + known) ** 3 - c * inv2
f = f.monic()
x0 = f.small_roots(X = 2**104)
for xs in x0:
    print(long_to_bytes(Integer(xs)))

b'Hello, World!'


# 3.5.2 Biết được rằng hai số nguyên tố lớn thỏa mãn thêm điều kiện khác

Bài toán

In [4]:
import Crypto
from Crypto.Util.number import bytes_to_long, long_to_bytes
import gmpy2
import random

N = int('0x91b9ed824be0348f88a44aab16a88d1393ee86db598a8380b30aed84cb06bae4836985de7d4aa95b958b912d5c29324e68bb1234b66afa6067f21aed432d3bd1', 16)
flag = b'Hello, World!'
m = bytes_to_long(flag)
e = 65537
c = pow(m, e, N)
print(f'c = {hex(c)}')

c = 0x5e50c10d9787df08c9b86c1e2480b9bbafa41b0dd8e4b3c9d2255fb4b87bd563e03c667525397884de4654c75557d56d16c656fa2e0724a58ea7efc44472e176


Lời giải

In [9]:
import Crypto
from Crypto.Util.number import bytes_to_long, long_to_bytes
import gmpy2
import random

sqrt = gmpy2.isqrt(N)

def find_sqrt_modulo_power_two(a2, e):
    if e <= 3 and (a2 & 7 == 1):
        return [2 * k + 1 for k in range(2**(e - 1))]
    else:
        if a2 & 7 != 1:
            return []
        else:
            a = [1, 3, 5, 7]
            pow = 16
            for exp in range(4, e + 1):
                roots = []
                target = a2 % pow
                for x in a:
                    if (x**2) % pow == target:
                        roots.append(x)
                        roots.append(x|(2**(exp - 1)))
                a.clear()
                a = roots[:]
                pow = pow << 1
            return a
        

def solve():    
    for half in range(120, 15, -1):
        for rand in range(half - 1, 7, -1):
            l = rand
            k = 256 - l - half
            a = (sqrt >> (256 - l))
            F.<x> = PolynomialRing(Zmod(N))

            for b in find_sqrt_modulo_power_two(N % (2**k), k):
                c = (a << (256 - l)) + b
                inv = gmpy2.invert(1 << k, N)
                f = x + int(c * inv)
                f = f.monic()
                x0 = f.small_roots(X=(2 ** half - 1), beta=0.44, epsilon=1/32)
                for xs in x0:
                    pcand = c + Integer(xs) * (1 << k)
                    p = Integer(pcand)
                    if N % pcand == 0:
                        print(f'l = {l}, k = {k}')
                        print(f'[+] p = {p}')
                        q = N//p
                        print(f'[+] q = {q}')
                        return p, q

In [10]:
p, q = solve()
phi = (p - 1) * (q - 1)
d = gmpy2.invert(e, phi)
m = pow(c, d, N)
print(long_to_bytes(int(m)))

l = 72, k = 64
[+] p = 87363061981042418024446851108683195130040255634985677544463653819040881563593
[+] q = 87363061981042418024415618260561587399224362984748717231642857324483861927881
b'Hello, World!'


# 3.6.1 Hai khóa bí mật chênh lệch chỉ vài đơn vị.

Bài toán

In [11]:
import Crypto
from Crypto.Util.number import bytes_to_long, long_to_bytes, getPrime
import gmpy2
import random

m = bytes_to_long(b'Hello, World!')
d1 = 65537

def genKey(d=6):
    while True:
        p = getPrime(1024)
        q = getPrime(1024)
        phi = (p - 1) * (q - 1)
        try:
            e1 = gmpy2.invert(d1, phi)
            e2 = gmpy2.invert(d1 + d, phi)
            return (p * q, e1, e2)
        except:
            continue

N, e1, e2 = genKey()
c = pow(m, e1, N)

print(f'''N = {hex(N)}
e1 = {hex(e1)}
e2 = {hex(e2)}
c = {hex(c)}''')

N = 0x9c1c57619c3f372044373e6a28d6532ea04b507b88f1c69cb91db2cc4c196aaf7c6c26bcc9c2fc4227ad6a370eb84edb4f32e2feef7e29ef57cff0ede7a9e314aca105833def322f09b83e9e8d30d5ddcc6508a16095a941d7ea296dbfa6053778fa7e0860eca8f8c65b861bd2d13790152f8258060f57fa3df59a6edf3e88c0bd8812ce7f42e9efeb3ce93fe8ff967c56d1283bc5e68ed2b418ea1fb3af3268fbf952c95425e79f523b5d7a3c928d7ca3232470fd3371ba757d9e8af36153e42c376c59739715b0032bd1cfa9d1fc47fee00099d8738e3c3e8102ab5fedc78b8ce56c20a21a24df82e6d94524d000e05f29d2ad55fef87985e68aa4d202283f
e1 = 0x6e3ba1043ef57072ab58ab0d015b1e0fad86a1052a55375ed164842db88ab607c67f03f7892cd33dcf90a790cb27201fc253f76ff7230a6f847d43ee6a084f5df8ec16dd6ead69ad475abf74ebd411207644dd8c3e8c4fa13bda90787bbcbab04a0fef50f1480ba4e7441775ac2ea9e266e34e51e3ad8ffd1725477084d93e6c6137578b1b0212d96baa32f8df090a4a05683c7ea2c7f2a99fce9565637d69a52c04483a19625737d45798aa769e1f1d23c791713e1cbb4dc11e58795151dcb61bee2e290a57a0a47636a45877d3d1d50e59f826e44f854b73b466234330333429902239e4e225d2f68803e0ca

Lời giải

In [12]:
mult = 6 * e1 * e2 + e2 - e1
g = gmpy2.gcd(mult, e1)
print(g)
d = gmpy2.invert(e1, mult//g)
print(d)
m = pow(c, d, N)
print(long_to_bytes(m))

5
23138938225697966961842153727294667488058344090019818051483379844956950257826463604995846982888078008333667771904929796160385977743075803376491316310107060989772663228963323337936579248917048813851802807903122785063796500062901512686187723278122629827567012178748892119397493135278714592136722235690129477081010791434909992369430255982721378196302518786827190304806177270818375553001028021787701007118792372272982933222427376973288878066309801749268179130989143817168865369671999190008067901731727939969850500071568161257537503596332041436386045478627819795054340162674820515279041811564156352370077749799867052958446740728647068863533143116914964889272134387670024737567837355834123317394403078224689012927427294128617162902291912296775259973161595630391910781719666154759883000493688506349346518827569262091962120643215238340356817714595230013789545049680557803201704572542242711997868252892898960142181130406587835532759590759174171296107622629681612598481488837121008406773279649388089376057419

# 3.6.2 Trường hợp khi số mũ và giá trị hàm phi-Euler không nguyên tố cùng nhau

Bài toán

In [11]:
import Crypto
from Crypto.Util.number import bytes_to_long, long_to_bytes, getPrime
import gmpy2
import random

m = bytes_to_long(b'Hello, World!')
e = 65537

def genKey(size):
    while True:
        p = getPrime(size)
        q = 2 * p + 1
        if gmpy2.is_prime(q):
            return p, q

p, q = genKey(1024)
N = p * q
c = pow(m, N, N)

print(f'''N = {hex(N)}
c = {hex(c)}''')

N = 0x1b3eb2c5a767f7cc729a8fed67627b9688f741852c14496320ffea94a5ae6f16d507960c605e1ca4bc7517853ed4bee75c64dd97107491fd7d45803f5835736a8620da4c7febbe225fbda617843891d387f2b4a54a4403dcd2b8dc2551dae87f4fb0ee08d73ea4b6e55318fcc1ea1336f63b67b2957a46b563fee1f4d9da67655240d29fd1a5893aeb1bda7280ae526bdaf0bb0c3c52a117dfc63eeb8b85c5c1104b727c9cc568f09c572a630c7e85550043b4360109d18e8c329368b1e86e0c6f3d7cefedc0e8afcd5f64706fa59d2921ce32dfd6d1a22937d2c411c473867770ab7f0d1c46c50d652375073b3e275865acb50e2a2ea5a70be3dc635f4873285
c = 0xaaf4c940c58f66103e910d25cfc24f52fc75908d21d7aead87381b563973742276d441ba98aea08e9c2404a66384c289087bfc03f7855be64b01103fd2d20b6e1315f3bb55a6f900a93602a8a5d4d0c3765accfb951245cf8027526350f777523eec14509d95d8a351d7e11cda897f19b2c674f83fd7f1585742e138204f708ae3487a67c31732eea11bade665a995afdbda7c9a13398ea962e6fba145f729de9796aa7aaaea1


Lời giải

In [12]:
p = gmpy2.isqrt(N * 8 + 1) >> 2
q = 2 * p + 1
phi = 2 * p * (p - 1)
d = gmpy2.invert(q, phi)
m = pow(c, d, p)
print(long_to_bytes(m))


b'Hello, World!'
